<a href="https://colab.research.google.com/github/Rishit-dagli/MIRNet-TFJS/blob/main/MIRNet_TFJS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> MIRNet TFJS </h1>

This notebook shows the the process of converting the `SavedModel` we built in the [prequel notebook](https://github.com/Rishit-dagli/MIRNet-TFJS/blob/main/MIRNet_Saved_Model.ipynb) for the **MIRNet** model as proposed by [Learning Enriched Features for Real Image Restoration and Enhancement](https://arxiv.org/pdf/2003.06792v2.pdf) by Zamir et al. This model is capable of enhancing low-light images upto a great extent.

<h2>A bit about the architecture</h2>

The MIRNet presents a novel architecture with the collective goals of maintaining high-resolution representations through the entire network, and
receiving strong contextual information from the low-resolution representations.

The core of this approach is a multi-scale residual block containing the following key elements:
- parallel multi-resolution convolution streams for extracting multi-scale features
- information exchange across the multi-resolution streams
- spatial and channel attention mechanisms for capturing contextual information
- attention based multi-scale feature aggregation.

![](https://i.imgur.com/VOzfG9a.png)
<p align="center">
<small>Framework of MIRNet</small>
</p>

The figure above shows the framework of the proposed network MIRNet that learns enriched feature representations for image restoration and enhancement.

<h2>Examples</h2>

<p align="center">
<img src="https://github.com/soumik12345/MIRNet/raw/master/assets/lol_results.gif"</img><br>
<small><a href="https://github.com/soumik12345/MIRNet/blob/master/assets/lol_results.gif">Source</a></small>
</p>

Model training code and pre-trained weights are provided by Soumik through [this repository](https://github.com/soumik12345/MIRNet/).

## Setup

Installing TensorFlow JS

In [1]:
!sudo pip install tensorflowjs

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 112kB 3.9MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


Downloading the TensorFlow `SavedModel` which we made in the [prequel notebook](https://github.com/Rishit-dagli/MIRNet-TFJS/blob/main/MIRNet_Saved_Model.ipynb)

In [6]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SyqUAkixPfdd1894GcILtmTY3owaUem_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SyqUAkixPfdd1894GcILtmTY3owaUem_" -O saved-model.tar.gz && rm -rf /tmp/cookies.txt

--2020-12-20 17:07:16--  https://docs.google.com/uc?export=download&confirm=axhL&id=1SyqUAkixPfdd1894GcILtmTY3owaUem_
Resolving docs.google.com (docs.google.com)... 108.177.125.102, 108.177.125.101, 108.177.125.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.125.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-3k-docs.googleusercontent.com/docs/securesc/m88g6hel12mvkfm4f18onshjdeivqv9k/mq5aj7eqd54h9pp60aag90p70pek0edn/1608483975000/11941728310014738790/13527291313208939176Z/1SyqUAkixPfdd1894GcILtmTY3owaUem_?e=download [following]
--2020-12-20 17:07:16--  https://doc-0k-3k-docs.googleusercontent.com/docs/securesc/m88g6hel12mvkfm4f18onshjdeivqv9k/mq5aj7eqd54h9pp60aag90p70pek0edn/1608483975000/11941728310014738790/13527291313208939176Z/1SyqUAkixPfdd1894GcILtmTY3owaUem_?e=download
Resolving doc-0k-3k-docs.googleusercontent.com (doc-0k-3k-docs.googleusercontent.com)... 64.233.189.132, 2404:6800:4008:c07::84
Connec

Let us take a look if the `saved-model.tar.gz` appears which we just downloaded.

In [7]:
!ls

sample_data  saved-model.tar.gz


We will now unzip the `saved-model.tar.gz` which we just downloaded.

In [10]:
saved_model_dir = "savedmodel" #@param {type:"string"}

import shutil
shutil.unpack_archive("saved-model.tar.gz", saved_model_dir)

### A couple of imports

In [25]:
from PIL import Image
import os
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np

## About the model

If you recall in the last notebook we downloaded pre-trained weights and our model looked like this:

![](https://i.imgur.com/Nn6jap7.jpg)

We will now be converting this model to the TFJS format.

## Convert to TFJS

First off this is the directory structure you would end up with after making the conversions:

```
tfjs
├── no-comp
|   └── model.json
|   └── ...
├── float-16
|   └── model.json
|   └── ...
├── uint-16
|   └── model.json
|   └── ...
├── uint-8
|   └── model.json
|   └── ...
```

### No Compression Model

In [16]:
!tensorflowjs_converter --input_format=keras_saved_model --metadata= savedmodel tfjs/no-comp

2020-12-20 17:27:28.828428: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-20 17:27:30.833580: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 17:27:30.834864: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-20 17:27:30.888209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-20 17:27:30.889181: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-12-20 17:27:30.889283: I tensorflow/stream_executor/platform/default/dso_lo

### Float 16 Model

In [18]:
!tensorflowjs_converter --input_format=keras_saved_model --metadata= --quantize_float16=* savedmodel tfjs/float-16

2020-12-20 17:35:04.799758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-20 17:35:06.788642: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 17:35:06.789798: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-20 17:35:06.807993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-20 17:35:06.808713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-12-20 17:35:06.808761: I tensorflow/stream_executor/platform/default/dso_lo

### UINT-16 Model

Quantizing to `float16` may provide better accuracy over 2 byte affine integer scaling (`uint16`)

In [21]:
!tensorflowjs_converter --input_format=keras_saved_model --metadata= --quantize_uint16=* savedmodel tfjs/uint-16

2020-12-20 17:40:32.129611: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-20 17:40:34.106746: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 17:40:34.107742: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-20 17:40:34.125978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-20 17:40:34.126605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-12-20 17:40:34.126650: I tensorflow/stream_executor/platform/default/dso_lo

### UINT-8 Model

1-byte affine quantization, `uint8` in most cases provides a 4x size reduction at the cost of accuracy

In [22]:
!tensorflowjs_converter --input_format=keras_saved_model --metadata= --quantize_uint8=* savedmodel tfjs/test

2020-12-20 17:44:35.559183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-20 17:44:37.335927: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 17:44:37.337095: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-20 17:44:37.354870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-20 17:44:37.355567: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-12-20 17:44:37.355623: I tensorflow/stream_executor/platform/default/dso_lo

## Comparing Model sizes

In [ ]:
model_type = "no-comp" #@param ["no-comp", "float-16", "uint-16", "uint-8"]

def get_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

In [ ]:
def model_size(model_dir):
    size = get_size(model_dir)
    print('Model size is %f MBs.' % (size / 1024.0 / 1024.0))

model_size("tfjs/" + model_type)

Model size is 139.255836 MBs.


As an example you can see here that the `float-16` model is almost half the size of the original model

In [ ]:
model_size("tfjs/float-16")

Model size is 69.984879 MBs.


And that is all for this notebook where we first dwonlaoded the model we built earlier, converted the model to TFJS format and finally compared model sizes for different compression techniques.